In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from linearmodels import PanelOLS
import plotly.express as px

sns.set_theme()

In [2]:
path = 'Datasets/' # To be changed if needed

Source GDP_PC_PPP: https://data360.worldbank.org/en/indicator/WB_WDI_NY_GDP_PCAP_PP_CD

In [11]:
gdp = pd.read_csv(path + 'WB_WDI_NY_GDP_PCAP_PP_CD.csv')
hdr = pd.read_excel(path+'hdr-data.xlsx') # Upload the HDR data

In [27]:
Areas = ['East Asia & Pacific', 'Europe & Central Asia', 'Latin America & Caribbean', 'Middle East and North Africa', 'North America', 'South Asia', 'Sub-Saharan Africa','World']

In [32]:
gdp[(gdp['TIME_PERIOD']==2022) & (gdp['REF_AREA_NAME'].isin(Areas))][['REF_AREA_NAME','OBS_VALUE']]

,REF_AREA_NAME,OBS_VALUE
7786,East Asia & Pacific,23272.948693
7790,Europe & Central Asia,46989.846943
7803,Latin America & Caribbean,20802.671634
7810,Middle East and North Africa,19862.160239
7814,North America,76436.683400
7821,South Asia,8538.424167
7823,Sub-Saharan Africa,4584.761596
7827,World,21631.625563


In [29]:
HDR = hdr.copy().drop(columns=['dimension','note','country']) # Drop unnecessary columns
HDR['year'] = pd.to_datetime(HDR['year'], format='%Y',errors='coerce').dt.year # Convert year to datetime format
HDR.rename(columns={'countryIsoCode':'CCode'},inplace=True) # Rename columns for consistency

indecies = HDR['indexCode'].unique() # Get unique Country codes
grouped_dfs = {} # Create a dictionary to store dataframes for each index

for code in indecies:
    grouped_dfs[code] = HDR[HDR['indexCode'] == code].drop(columns=['indexCode', 'index']) # Drop unnecessary columns

GII = grouped_dfs.get('GII') # Get the GII dataframe
HDI = grouped_dfs.get('HDI') # Get the HDI dataframe
GDI = grouped_dfs.get('GDI') # Get the GDI dataframe
IHDI = grouped_dfs.get('IHDI') # Get the IHDI dataframe
PHDI = grouped_dfs.get('PHDI') # Get the PHDI dataframe
MPI = grouped_dfs.get('MPI') # Get the MPI dataframe

In [31]:
GII['CCode'].unique() # Get unique country codes

array(['AFG', 'AGO', 'ALB', 'AND', 'ARE', 'ARG', 'ARM', 'ATG', 'AUS',
       'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR',
       'BHS', 'BIH', 'BLR', 'BLZ', 'BOL', 'BRA', 'BRB', 'BRN', 'BTN',
       'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CIV', 'CMR', 'COD',
       'COG', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU',
       'DJI', 'DMA', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP',
       'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'FSM', 'GAB', 'GBR', 'GEO',
       'GHA', 'GIN', 'GMB', 'GNB', 'GNQ', 'GRC', 'GRD', 'GTM', 'GUY',
       'HKG', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN',
       'IRQ', 'ISL', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN',
       'KGZ', 'KHM', 'KIR', 'KNA', 'KOR', 'KWT', 'LAO', 'LBN', 'LBR',
       'LBY', 'LCA', 'LIE', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAR',
       'MCO', 'MDA', 'MDG', 'MDV', 'MEX', 'MHL', 'MKD', 'MLI', 'MLT',
       'MMR', 'MNE', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM',
       'NER', 'NGA',

In [33]:
gdp

,STRUCTURE,STRUCTURE_ID,ACTION,FREQ_ID,FREQ_NAME,REF_AREA_ID,REF_AREA_NAME,INDICATOR_ID,INDICATOR_NAME,SEX_ID,...,DATA_SOURCE_NAME,UNIT_TYPE_ID,UNIT_TYPE_NAME,TIME_FORMAT_ID,TIME_FORMAT_NAME,COMMENT_OBS,OBS_STATUS_ID,OBS_STATUS_NAME,OBS_CONF_ID,OBS_CONF_NAME
0,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,AFE,Africa Eastern and Southern,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
1,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,AFW,Africa Western and Central,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
2,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,ARB,Arab World,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
3,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,CSS,Caribbean small states,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
4,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,CEB,Central Electricity Board (CEB),WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8261,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,VUT,Vanuatu,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
8262,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,VNM,Viet Nam,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
8263,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,PSE,West Bank and Gaza,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public
8264,datastructure,WB.DATA360:DS_DATA360(1.2),I,A,Annual,ZMB,Zambia,WB_WDI_NY_GDP_PCAP_PP_CD,"GDP per capita, PPP (current international $)",_T,...,World Development Indicators (WDI),CUR,Currency,P1Y,Annual,NaN,A,Normal value,PU,Public


In [37]:
df = gdp[['REF_AREA_ID','TIME_PERIOD','OBS_VALUE']]
df = df[df['TIME_PERIOD'] == 2022] # Filter for the year 2022

In [47]:
fig = px.choropleth(
    df,
    locations="REF_AREA_ID",           # Colonna dei nomi dei paesi
    locationmode="ISO-3",   # Usa i nomi dei paesi
    color="OBS_VALUE",                  # Colonna con i valori numerici
    color_continuous_scale="Blues", # Scegli una scala di colori
    title="GDP per capita, PPP (current international $) - 2022"
)
fig.update_geos(
    showframe=False,
    showcoastlines=True,
    showland=True,
    showocean=True,
    landcolor='darkgray',
    oceancolor="#E0FFFF", 
    projection_type="natural earth"
)

fig.update_layout(
    margin={"r":0,"t":40,"l":10,"b":10},
    coloraxis_colorbar=dict(
        title="(%)",
        ticks="outside",
        ticklen=5,
        tickcolor='darkgray'
    ),
    font=dict(
        family="Helvetica",
        size=14,
        color="black"
    ),
    title=dict(
        text="GDP per capita, PPP (current international $) - 2022",
        x=0.5,  # Centra il titolo
        xanchor="center",  # Assicura che il titolo sia ancorato al centro
        yanchor="top"      # Mantiene il titolo nella parte superiore
    ),
    width=1200,
    height=600,
)
fig.update_traces(
    marker_line_width=0.5, 
    marker_line_color="black",
    autocolorscale=False,
    colorbar_title="($)"
)



fig.show()